In [1]:
# Import necessary libraries
import requests       # to fetch data from the web API
import pandas as pd   # to organize and store the data

# Step 1: Define a list of drug names we are interested in
drug_names = ['paracetamol', 'ibuprofen', 'celecoxib']

# Step 2: Create an empty list to collect drug information
drug_data = []

# Step 3: Loop through each drug to query ChEMBL and retrieve SMILES
for drug in drug_names:
    # Use ChEMBL API to search for the drug and get its ChEMBL ID
    url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/search?q={drug}"
    response = requests.get(url, headers={"Accept": "application/json"})

    # If the search is successful (status code 200)
    if response.status_code == 200:
        results = response.json()
        # Get the first hit from the search result
        if results['molecules']:
            chembl_id = results['molecules'][0]['molecule_chembl_id']

            # Use the ChEMBL ID to get molecule details (like SMILES)
            mol_url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/{chembl_id}.json"
            mol_response = requests.get(mol_url)

            if mol_response.status_code == 200:
                mol_data = mol_response.json()

                # Extract canonical SMILES string
                smiles = mol_data.get('molecule_structures', {}).get('canonical_smiles', 'NA')

                # Append results to the list
                drug_data.append({'drug': drug, 'chembl_id': chembl_id, 'smiles': smiles})
    else:
        print(f"Failed to retrieve data for {drug}")

# Step 4: Convert collected data into a pandas DataFrame
df_smiles = pd.DataFrame(drug_data)

# Step 5: Display the results
print("SMILES DataFrame:")
print(df_smiles)

# Optional: Save to CSV
df_smiles.to_csv("drug_smiles.csv", index=False)


SMILES DataFrame:
          drug  chembl_id                                             smiles
0  paracetamol  CHEMBL112                                 CC(=O)Nc1ccc(O)cc1
1    ibuprofen  CHEMBL521                         CC(C)Cc1ccc(C(C)C(=O)O)cc1
2    celecoxib  CHEMBL118  Cc1ccc(-c2cc(C(F)(F)F)nn2-c2ccc(S(N)(=O)=O)cc2...


In [9]:
# Sesión global con reintentos y User-Agent
import requests, time
from requests.adapters import HTTPAdapter
try:
    from urllib3.util.retry import Retry
except Exception:
    # Compatibilidad si cambia la ruta en tu versión
    from urllib3.util import Retry  # type: ignore

SESSION = requests.Session()
SESSION.headers.update({"User-Agent": "Sandra-Capstone/1.0"})
retries = Retry(
    total=5,
    connect=5,
    read=5,
    backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=frozenset(["GET"])
)
adapter = HTTPAdapter(max_retries=retries)
SESSION.mount("https://", adapter)
SESSION.mount("http://", adapter)

# Intenta usar el bundle de certificados si está presente
VERIFY = True
try:
    import certifi
    VERIFY = certifi.where()
except Exception:
    pass

def get_fasta_seq(uniprot_id, timeout=15):
    bases = [
        f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta",
        f"https://www.uniprot.org/uniprotkb/{uniprot_id}.fasta",
        f"https://legacy.uniprot.org/uniprotkb/{uniprot_id}.fasta",
    ]
    last_err = None
    for url in bases:
        try:
            r = SESSION.get(url, timeout=timeout, verify=VERIFY)
            if r.status_code == 200 and r.text and ">" in r.text:
                return "".join([ln.strip() for ln in r.text.splitlines() if not ln.startswith(">")])
        except requests.exceptions.SSLError as e:
            last_err = e
            time.sleep(0.8)
        except requests.exceptions.RequestException as e:
            last_err = e
            time.sleep(0.8)
    # Si no se pudo, devolvemos None; el pipeline usará un fallback one hot.
    print(f"Advertencia: no se pudo descargar FASTA de {uniprot_id}. Seguiremos con fallback. Último error: {last_err}")
    return None



In [10]:
# Capstone COX1 vs COX2 - versión simple con reintentos SSL y fallback de proteína
# Requisitos: requests, pandas, numpy, scikit-learn

import requests, time, re
import pandas as pd
import numpy as np
from requests.adapters import HTTPAdapter
try:
    from urllib3.util.retry import Retry
except Exception:
    from urllib3.util import Retry  # type: ignore

# 1) Sesión robusta
SESSION = requests.Session()
SESSION.headers.update({"User-Agent": "Sandra-Capstone/1.0"})
retries = Retry(total=5, connect=5, read=5, backoff_factor=0.6,
                status_forcelist=[429,500,502,503,504],
                allowed_methods=frozenset(["GET"]))
adapter = HTTPAdapter(max_retries=retries)
SESSION.mount("https://", adapter)
SESSION.mount("http://", adapter)

VERIFY = True
try:
    import certifi
    VERIFY = certifi.where()
except Exception:
    pass

def http_get(url, params=None, timeout=20):
    r = SESSION.get(url, params=params, headers={"Accept":"application/json"}, timeout=timeout, verify=VERIFY)
    r.raise_for_status()
    return r

# 2) IDs y parámetros
CHEMBL_COX1 = "CHEMBL2095188"
CHEMBL_COX2 = "CHEMBL2094253"
UNIPROT_COX1 = "P23219"
UNIPROT_COX2 = "P35354"
STD_TYPE = "IC50"
STD_UNITS = "nM"
RANDOM_STATE = 42

# 3) UniProt FASTA con fallback
def get_fasta_seq(uniprot_id, timeout=15):
    bases = [
        f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta",
        f"https://www.uniprot.org/uniprotkb/{uniprot_id}.fasta",
        f"https://legacy.uniprot.org/uniprotkb/{uniprot_id}.fasta",
    ]
    last_err = None
    for url in bases:
        try:
            r = SESSION.get(url, timeout=timeout, verify=VERIFY)
            if r.status_code == 200 and r.text and ">" in r.text:
                return "".join([ln.strip() for ln in r.text.splitlines() if not ln.startswith(">")])
        except requests.exceptions.SSLError as e:
            last_err = e; time.sleep(0.8)
        except requests.exceptions.RequestException as e:
            last_err = e; time.sleep(0.8)
    print(f"Advertencia: no se pudo descargar FASTA de {uniprot_id}. Fallback one hot. Último error: {last_err}")
    return None

seq_cox1 = get_fasta_seq(UNIPROT_COX1)
seq_cox2 = get_fasta_seq(UNIPROT_COX2)

AA = list("ACDEFGHIKLMNPQRSTVWY")
def aa_comp(seq):
    n = len(seq)
    return np.array([seq.count(a)/n for a in AA], dtype=float)

# Si no hay FASTA, usa one hot de proteína como fallback
if seq_cox1 and seq_cox2:
    aa_cox1 = aa_comp(seq_cox1)
    aa_cox2 = aa_comp(seq_cox2)
    PROT_FEATURES = "aa"   # usaremos composición AA
else:
    aa_cox1 = None
    aa_cox2 = None
    PROT_FEATURES = "onehot"  # usaremos one hot

# 4) Actividades de ChEMBL (paginación simple + reintentos)
def get_activities(target_id, max_pages=10, page_size=200):
    base = "https://www.ebi.ac.uk/chembl/api/data/activity.json"
    rows = []
    for p in range(max_pages):
        params = {
            "target_chembl_id": target_id,
            "standard_type": STD_TYPE,
            "standard_units": STD_UNITS,
            "limit": page_size,
            "offset": p*page_size
        }
        try:
            r = http_get(base, params=params, timeout=25)
        except requests.exceptions.RequestException as e:
            print(f"Advertencia: fallo en página {p} para {target_id}: {e}")
            break
        data = r.json().get("activities", [])
        if not data: break
        rows.extend(data)
        time.sleep(0.1)
    return pd.DataFrame(rows)

df1 = get_activities(CHEMBL_COX1)
df2 = get_activities(CHEMBL_COX2)

# 5) Limpieza básica y etiqueta
for df in (df1, df2):
    df["standard_value"] = pd.to_numeric(df["standard_value"], errors="coerce")

def clean(df):
    cols = ["molecule_chembl_id","target_chembl_id","standard_value","canonical_smiles"]
    for c in cols:
        if c not in df.columns: df[c] = None
    df = df.dropna(subset=["standard_value"])
    df = df[df["standard_value"] > 0]
    df = (df.groupby(["molecule_chembl_id","target_chembl_id"], as_index=False)
            .agg({"standard_value":"median","canonical_smiles":"first"}))
    return df

df1 = clean(df1); df1["protein"] = "COX1"; df1["label"] = 0
df2 = clean(df2); df2["protein"] = "COX2"; df2["label"] = 1
pairs = pd.concat([df1, df2], ignore_index=True)

# 6) SMILES faltantes
def fetch_smiles(mol_id):
    url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/{mol_id}.json"
    try:
        r = http_get(url, timeout=20)
        j = r.json().get("molecule_structures", {})
        return j.get("canonical_smiles", None)
    except requests.exceptions.RequestException:
        return None

mask_missing = pairs["canonical_smiles"].isna() | (pairs["canonical_smiles"]=="")
need = pairs.loc[mask_missing, "molecule_chembl_id"].dropna().unique().tolist()
smi_cache = {}
for m in need:
    smi_cache[m] = fetch_smiles(m)
    time.sleep(0.05)
pairs.loc[mask_missing, "canonical_smiles"] = pairs.loc[mask_missing, "molecule_chembl_id"].map(smi_cache)
pairs = pairs.dropna(subset=["canonical_smiles"]).reset_index(drop=True)

print("pairs shape:", pairs.shape)

# 7) Huellas simples por SMILES (3-gramas con hashing) - sin RDKit
def smiles_to_ngram(smi, nbits=1024):
    s = re.sub(r"\s+", "", smi or "")
    grams = [s[i:i+3] for i in range(max(1, len(s)-2))]
    v = np.zeros(nbits, dtype=np.int8)
    for g in grams:
        v[hash(g) % nbits] = 1
    return v

uniq = pairs[["molecule_chembl_id","canonical_smiles"]].drop_duplicates().reset_index(drop=True)
fp_mat = np.vstack([smiles_to_ngram(s) for s in uniq["canonical_smiles"]])
fp_df = pd.DataFrame(fp_mat, columns=[f"fp_{i}" for i in range(fp_mat.shape[1])])
fp_df.insert(0, "molecule_chembl_id", uniq["molecule_chembl_id"].values)

# 8) Rasgos de proteína
if PROT_FEATURES == "aa":
    prot_df = pd.DataFrame({
        "protein":["COX1","COX2"],
        **{f"aa_{aa}":[aa_cox1[i], aa_cox2[i]] for i, aa in enumerate(AA)}
    })
else:
    # Fallback one hot si no hubo FASTA
    prot_df = pd.DataFrame({
        "protein":["COX1","COX2"],
        "prot_COX1":[1,0],
        "prot_COX2":[0,1],
    })

# 9) Tabla final y modelo
dfm = pairs.merge(fp_df, on="molecule_chembl_id", how="left").merge(prot_df, on="protein", how="left")
feat_cols = [c for c in dfm.columns if c.startswith("fp_")]
if PROT_FEATURES == "aa":
    feat_cols += [c for c in dfm.columns if c.startswith("aa_")]
else:
    feat_cols += ["prot_COX1","prot_COX2"]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

X = dfm[feat_cols].values
y = dfm["label"].values
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

clf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)
clf.fit(Xtr, ytr)
yp = clf.predict(Xte)
ypb = clf.predict_proba(Xte)[:,1]

print("dfm shape:", dfm.shape)
print("protein features:", PROT_FEATURES)
print("accuracy:", round(accuracy_score(yte, yp), 3))
print("precision:", round(precision_score(yte, yp), 3))
print("recall:", round(recall_score(yte, yp), 3))
print("f1:", round(f1_score(yte, yp), 3))
print("roc_auc:", round(roc_auc_score(yte, ypb), 3))

# 10) Guardados mínimos
pairs.to_csv("chembl_pairs_ic50.csv", index=False)
dfm.to_csv("pcm_dataset.csv", index=False)


pairs shape: (790, 6)


ModuleNotFoundError: No module named 'scipy.sparse._sparsetools'

In [ ]:
# FINAL CODE #1

# Step 1: Download and clean activities from ChEMBL for COX1 and COX2
# Required: requests, pandas

import time
import pandas as pd
import requests

# Session with retries
from requests.adapters import HTTPAdapter
try:
    from urllib3.util.retry import Retry
except Exception:
    from urllib3.util import Retry  # compatibility

SESSION = requests.Session()
SESSION.headers.update({"User-Agent": "Sandra-Capstone/1.0", "Accept": "application/json"})
retries = Retry(
    total=5, connect=5, read=5, backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=frozenset(["GET"])
)
adapter = HTTPAdapter(max_retries=retries)
SESSION.mount("https://", adapter)
SESSION.mount("http://", adapter)

def http_get_json(url, params=None, timeout=25):
    r = SESSION.get(url, params=params, timeout=timeout)
    r.raise_for_status()
    return r.json()

# Project parameters
CHEMBL_COX1 = "CHEMBL2095188"
CHEMBL_COX2 = "CHEMBL2094253"
STD_TYPE = "IC50"
STD_UNITS = "nM"

# Paginated download of activities
def get_activities(target_id, max_pages=20, page_size=200):
    base = "https://www.ebi.ac.uk/chembl/api/data/activity.json"
    rows = []
    for p in range(max_pages):
        params = {
            "target_chembl_id": target_id,
            "standard_type": STD_TYPE,
            "standard_units": STD_UNITS,
            "limit": page_size,
            "offset": p * page_size
        }
        try:
            data = http_get_json(base, params=params)
        except requests.exceptions.RequestException as e:
            print(f"notice: failed page {p} for {target_id}: {e}")
            break
        items = data.get("activities", [])
        if not items:
            break
        rows.extend(items)
        time.sleep(0.1)
    return pd.DataFrame(rows)

print("downloading activities...")
df1 = get_activities(CHEMBL_COX1)
df2 = get_activities(CHEMBL_COX2)
print("raw records:", len(df1), len(df2))

# Basic cleaning and labeling
for df in (df1, df2):
    df["standard_value"] = pd.to_numeric(df["standard_value"], errors="coerce")

def clean(df):
    cols = ["molecule_chembl_id", "target_chembl_id", "standard_value", "standard_relation", "canonical_smiles"]
    for c in cols:
        if c not in df.columns:
            df[c] = None
    # For now we only keep relation equal to avoid inequalities
    df = df[df["standard_relation"].fillna("=") == "="]
    df = df.dropna(subset=["standard_value"])
    df = df[df["standard_value"] > 0]
    # Collapse duplicates by molecule and target using median IC50
    df = (
        df.groupby(["molecule_chembl_id", "target_chembl_id"], as_index=False)
          .agg({"standard_value": "median", "canonical_smiles": "first"})
    )
    return df

df1 = clean(df1); df1["protein"] = "COX1"; df1["label"] = 0
df2 = clean(df2); df2["protein"] = "COX2"; df2["label"] = 1
pairs = pd.concat([df1, df2], ignore_index=True)

# Ensure SMILES if missing
def fetch_smiles(mol_id):
    url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/{mol_id}.json"
    try:
        j = http_get_json(url)
    except requests.exceptions.RequestException:
        return None
    return j.get("molecule_structures", {}).get("canonical_smiles", None)

mask_missing = pairs["canonical_smiles"].isna() | (pairs["canonical_smiles"] == "")
need = pairs.loc[mask_missing, "molecule_chembl_id"].dropna().unique().tolist()
cache = {}
for m in need:
    cache[m] = fetch_smiles(m)
    time.sleep(0.05)
pairs.loc[mask_missing, "canonical_smiles"] = pairs.loc[mask_missing, "molecule_chembl_id"].map(cache)

# Discard entries without SMILES
pairs = pairs.dropna(subset=["canonical_smiles"]).reset_index(drop=True)

# Small control views
print("total pairs:", len(pairs))
print("unique molecules:", pairs["molecule_chembl_id"].nunique())
print("by class:", pairs["protein"].value_counts().to_dict())
print(pairs.head(5))

# Save for the next step
pairs.to_csv("step1_pairs_ic50.csv", index=False)
print("done: step1_pairs_ic50.csv")

### Step 1 download IC50 activity records for COX1 and COX2, clean them, and save molecule target pairs with SMILES. 
### We downloaded IC50 activity data from ChEMBL for COX1 and COX2, cleaned it, and kept only valid values where the relation was “=”. 
# We removed duplicates by taking the median IC50 for each molecule–target pair, added SMILES if they were missing, and labeled COX1 as 0 and COX2 as 1. 
# The results show 580 total pairs (449 COX1 and 131 COX2). This means most of the data is for COX1, and each molecule is linked to only one target in this dataset.


downloading activities...
raw records: 639 191
total pairs: 580
unique molecules: 580
by class: {'COX1': 449, 'COX2': 131}
  molecule_chembl_id target_chembl_id  standard_value  \
0       CHEMBL102714    CHEMBL2095188            54.5   
1      CHEMBL1076156    CHEMBL2095188         15000.0   
2      CHEMBL1081243    CHEMBL2095188          1400.0   
3      CHEMBL1081244    CHEMBL2095188          7000.0   
4      CHEMBL1081557    CHEMBL2095188         38000.0   

                                    canonical_smiles protein  label  
0     Cn1cc(C2=C(c3ccc(Cl)cc3Cl)C(=O)NC2=O)c2ccccc21    COX1      0  
1  CN1C(=O)/C(=C/c2ccc3c(c2)OCO3)N=C1NCC/N=C/c1cc...    COX1      0  
2          CN1C(=O)/C(=C/c2ccc3c(c2)OCO3)N=C1NCCN.Cl    COX1      0  
3  CN1C(=O)/C(=C/c2ccc3c(c2)OCO3)N=C1NCC/N=C/c1cc...    COX1      0  
4  CN1C(=O)/C(=C/c2ccc3c(c2)OCO3)N=C1NCC/N=C/c1cc...    COX1      0  
done: step1_pairs_ic50.csv


In [ ]:
# Step 2: Protein features for COX1 and COX2
# Requirements: requests, pandas, numpy

import requests, time
import pandas as pd
import numpy as np
from requests.adapters import HTTPAdapter
try:
    from urllib3.util.retry import Retry
except Exception:
    from urllib3.util import Retry  # compatibility

# Reuse a robust session
SESSION = requests.Session()
SESSION.headers.update({"User-Agent": "Sandra-Capstone/1.0"})
retries = Retry(
    total=5, connect=5, read=5, backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=frozenset(["GET"])
)
adapter = HTTPAdapter(max_retries=retries)
SESSION.mount("https://", adapter)
SESSION.mount("http://", adapter)

VERIFY = True
try:
    import certifi
    VERIFY = certifi.where()
except Exception:
    pass

# UniProt IDs
UNIPROT_COX1 = "P23219"
UNIPROT_COX2 = "P35354"

# Try multiple UniProt endpoints for reliability
def get_fasta_seq(uniprot_id, timeout=15):
    urls = [
        f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta",
        f"https://www.uniprot.org/uniprotkb/{uniprot_id}.fasta",
        f"https://legacy.uniprot.org/uniprotkb/{uniprot_id}.fasta",
    ]
    last_err = None
    for url in urls:
        try:
            r = SESSION.get(url, timeout=timeout, verify=VERIFY)
            if r.status_code == 200 and ">" in r.text:
                return "".join([ln.strip() for ln in r.text.splitlines() if not ln.startswith(">")])
        except requests.exceptions.RequestException as e:
            last_err = e
            time.sleep(0.8)
    print(f"Warning: Could not get FASTA for {uniprot_id}. Last error: {last_err}")
    return None

# Get sequences
seq_cox1 = get_fasta_seq(UNIPROT_COX1)
seq_cox2 = get_fasta_seq(UNIPROT_COX2)

# Amino acid composition
AA = list("ACDEFGHIKLMNPQRSTVWY")
def aa_comp(seq):
    n = len(seq)
    return np.array([seq.count(a) / n for a in AA], dtype=float)

# Build dataframe
if seq_cox1 and seq_cox2:
    aa_cox1 = aa_comp(seq_cox1)
    aa_cox2 = aa_comp(seq_cox2)
    prot_df = pd.DataFrame({
        "protein": ["COX1", "COX2"],
        **{f"aa_{aa}": [aa_cox1[i], aa_cox2[i]] for i, aa in enumerate(AA)}
    })
    feature_type = "amino_acid_composition"
else:
    prot_df = pd.DataFrame({
        "protein": ["COX1", "COX2"],
        "prot_COX1": [1, 0],
        "prot_COX2": [0, 1]
    })
    feature_type = "one_hot"

print(f"Protein feature type: {feature_type}")
print(prot_df)

# Save
prot_df.to_csv("step2_protein_features.csv", index=False)
print("Saved: step2_protein_features.csv")

### Get the amino acid sequences for COX1 and COX2 from UniProt and turn each sequence into a 20 feature vector of amino acid composition.
### We got the amino acid sequences for COX1 and COX2 from UniProt and calculated the fraction of each of the 20 amino acids in the sequence. 
# This gives a simple numeric profile for each protein. 
# The results show, for example, that COX1 has 7.51% glycine, and COX2 has 6.13% glycine. These values will be used later as protein features in the PCM model.


Protein feature type: amino_acid_composition
  protein      aa_A      aa_C      aa_D      aa_E      aa_F      aa_G  \
0    COX1  0.041736  0.021703  0.040067  0.060100  0.065109  0.075125   
1    COX2  0.051325  0.021523  0.043046  0.059603  0.062914  0.061258   

       aa_H      aa_I      aa_K  ...      aa_M      aa_N      aa_P      aa_Q  \
0  0.030050  0.041736  0.041736  ...  0.030050  0.031720  0.076795  0.045075   
1  0.031457  0.056291  0.056291  ...  0.024834  0.048013  0.066225  0.051325   

       aa_R      aa_S      aa_T      aa_V      aa_W      aa_Y  
0  0.055092  0.055092  0.050083  0.053422  0.016694  0.045075  
1  0.044702  0.057947  0.056291  0.057947  0.009934  0.044702  

[2 rows x 21 columns]
Saved: step2_protein_features.csv


In [ ]:
# Step 3: Ligand features from SMILES (temporary n-gram method)
# Requirements: pandas, numpy, re

import pandas as pd
import numpy as np
import re

# Load cleaned pairs from Step 1
pairs = pd.read_csv("step1_pairs_ic50.csv")
print("Loaded pairs:", pairs.shape)

# Function: convert SMILES to hashed n-gram vector
def smiles_to_ngram(smi, nbits=1024):
    s = re.sub(r"\s+", "", smi or "")
    grams = [s[i:i+3] for i in range(max(1, len(s)-2))]
    v = np.zeros(nbits, dtype=np.int8)
    for g in grams:
        v[hash(g) % nbits] = 1
    return v

# Get unique molecules
uniq = pairs[["molecule_chembl_id", "canonical_smiles"]].drop_duplicates().reset_index(drop=True)
print("Unique molecules:", uniq.shape)

# Build feature matrix
fp_mat = np.vstack([smiles_to_ngram(s) for s in uniq["canonical_smiles"]])
fp_df = pd.DataFrame(fp_mat, columns=[f"fp_{i}" for i in range(fp_mat.shape[1])])
fp_df.insert(0, "molecule_chembl_id", uniq["molecule_chembl_id"].values)

# Save features
fp_df.to_csv("step3_ligand_features.csv", index=False)
print("Saved: step3_ligand_features.csv")

### turn each SMILES into a 1024 bit hashed ngram fingerprint without RDKit.
### Loaded pairs: (580, 6) and Unique molecules: (580, 2) shows one molecule per pair in this cleaned set.

### We converted each molecule’s SMILES string into a 1024-bit vector using hashed 3-character n-grams (a simple way to represent molecular structure). 
# This creates ligand fingerprints without needing RDKit. The results show 580 unique molecules, each with its own fingerprint, saved for later steps.


Loaded pairs: (580, 6)
Unique molecules: (580, 2)
Saved: step3_ligand_features.csv


In [17]:
import numpy, scipy, sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
print("OK:", numpy.__version__, scipy.__version__, sklearn.__version__)


OK: 2.3.2 1.16.1 1.7.1


In [25]:
# NOT CORRECT, DATA TOO PERFECT
# Step 4: Merge datasets and train model
# Requirements: pandas, numpy, scikit-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load all datasets
pairs = pd.read_csv("step1_pairs_ic50.csv")
prot_df = pd.read_csv("step2_protein_features.csv")
lig_df = pd.read_csv("step3_ligand_features.csv")

print("Pairs:", pairs.shape)
print("Protein features:", prot_df.shape)
print("Ligand features:", lig_df.shape)

# Merge ligand features
merged = pairs.merge(lig_df, on="molecule_chembl_id", how="left")
# Merge protein features
merged = merged.merge(prot_df, on="protein", how="left")

print("Merged dataset:", merged.shape)

# Define features
feature_cols = [c for c in merged.columns if c.startswith("fp_") or c.startswith("aa_") or c.startswith("prot_")]
X = merged[feature_cols].values
y = merged["label"].values

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

# Metrics
metrics = {
    "accuracy": round(accuracy_score(y_test, y_pred), 3),
    "precision": round(precision_score(y_test, y_pred), 3),
    "recall": round(recall_score(y_test, y_pred), 3),
    "f1": round(f1_score(y_test, y_pred), 3),
    "roc_auc": round(roc_auc_score(y_test, y_prob), 3)
}
print("Metrics:", metrics)

# Save merged dataset and metrics
merged.to_csv("step4_merged_dataset.csv", index=False)
pd.DataFrame([metrics]).to_csv("step4_model_metrics.csv", index=False)
print("Saved: step4_merged_dataset.csv, step4_model_metrics.csv")


Pairs: (580, 6)
Protein features: (2, 21)
Ligand features: (580, 1025)
Merged dataset: (580, 1050)
Metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'roc_auc': 1.0}
Saved: step4_merged_dataset.csv, step4_model_metrics.csv


In [ ]:
# Step 4: Single-target activity classification for COX2 using pChEMBL
# Requirements: pandas, numpy, scikit-learn, re

import pandas as pd
import numpy as np
import re

from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)

# ### 1) Load pChEMBL pairs and keep only COX2
pairs_p = pd.read_csv("step1b_pairs_pchembl.csv")   # columns: molecule_chembl_id, protein, pchembl_value, canonical_smiles
cox2 = pairs_p[pairs_p["protein"] == "COX2"].copy()

# Keep one pChEMBL per molecule (max potency)
cox2 = (cox2.groupby(["molecule_chembl_id"], as_index=False)
             .agg({"pchembl_value": "max", "canonical_smiles": "first"}))

print("COX2 molecules (with pChEMBL):", cox2.shape[0])

# ### 2) Define activity labels: Active >= 6.0 (≈ ≤ 1 μM), else Inactive
THRESH = 6.0
cox2["label_active"] = (cox2["pchembl_value"] >= THRESH).astype(int)
print("Class balance:", cox2["label_active"].value_counts().to_dict())

# ### 3) Create ligand features (hashed SMILES n-grams; RDKit-free)
def smiles_to_ngram(smi, nbits=1024):
    s = re.sub(r"\s+", "", smi or "")
    grams = [s[i:i+3] for i in range(max(1, len(s)-2))]
    v = np.zeros(nbits, dtype=np.int8)
    for g in grams:
        v[hash(g) % nbits] = 1
    return v

uniq = cox2[["molecule_chembl_id", "canonical_smiles"]].drop_duplicates().reset_index(drop=True)
fp_mat = np.vstack([smiles_to_ngram(s) for s in uniq["canonical_smiles"]])
fp_cols = [f"fp_{i}" for i in range(fp_mat.shape[1])]
fp_df = pd.DataFrame(fp_mat, columns=fp_cols)
fp_df.insert(0, "molecule_chembl_id", uniq["molecule_chembl_id"].values)

# Merge fingerprints into COX2 table
data = cox2.merge(fp_df, on="molecule_chembl_id", how="inner")

# ### 4) Build X, y and group split by molecule
feature_cols = [c for c in data.columns if c.startswith("fp_")]
X = data[feature_cols].values
y = data["label_active"].values
groups = data["molecule_chembl_id"].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr_idx, te_idx = next(gss.split(X, y, groups))
Xtr, Xte = X[tr_idx], X[te_idx]
ytr, yte = y[tr_idx], y[te_idx]

# ### 5) Train a simple Random Forest
clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
clf.fit(Xtr, ytr)
yp = clf.predict(Xte)
ypb = clf.predict_proba(Xte)[:, 1]

# ### 6) Metrics + confusion matrix
tn, fp, fn, tp = confusion_matrix(yte, yp, labels=[0,1]).ravel()
metrics = {
    "accuracy": round(accuracy_score(yte, yp), 3),
    "precision": round(precision_score(yte, yp), 3),
    "recall": round(recall_score(yte, yp), 3),
    "f1": round(f1_score(yte, yp), 3),
    "roc_auc": round(roc_auc_score(yte, ypb), 3) if len(np.unique(yte)) > 1 else float("nan"),
    "n_train": int(len(ytr)),
    "n_test": int(len(yte)),
    "confusion_matrix": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
    "threshold_pchembl_active": THRESH
}
print("COX2 activity metrics (group split by molecule):", metrics)

# ### 7) Save final dataset and metrics
out = data.copy()
out["split"] = "train"
out.loc[te_idx, "split"] = "test"
out.to_csv("step4d_cox2_activity_dataset.csv", index=False)
pd.DataFrame([metrics]).to_csv("step4d_metrics.csv", index=False)
print("Saved: step4d_cox2_activity_dataset.csv, step4d_metrics.csv")

### This is a single target COX2 classifier that predicts active vs inactive from ligand features only
### We built a COX2-only classification model using pChEMBL values to label molecules as active (≥6.0) or inactive. 
# We split the data so the same molecule could not be in both train and test sets. The results show the model is 72% accurate, with high precision (83%) but lower recall (56%), meaning it’s good at avoiding false positives but misses many actives.



COX2 molecules (with pChEMBL): 87
Class balance: {0: 48, 1: 39}
COX2 activity metrics (group split by molecule): {'accuracy': 0.722, 'precision': 0.833, 'recall': 0.556, 'f1': 0.667, 'roc_auc': 0.71, 'n_train': 69, 'n_test': 18, 'confusion_matrix': {'tn': 8, 'fp': 1, 'fn': 4, 'tp': 5}, 'threshold_pchembl_active': 6.0}
Saved: step4d_cox2_activity_dataset.csv, step4d_metrics.csv


In [ ]:
# STEP 5: Feature Importance and Interpretability for COX2 Model (robust, headless-safe)
# Requirements: pandas, numpy, matplotlib, scikit-learn

import os
import pandas as pd
import numpy as np

# Use a non-interactive backend to avoid GUI issues
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

# ### 1) Load dataset
in_path = "step4d_cox2_activity_dataset.csv"
if not os.path.exists(in_path):
    raise FileNotFoundError(f"Expected file '{in_path}' not found. Run Step 4d first.")

data = pd.read_csv(in_path)

# ### 2) Select features and labels
feature_cols = [c for c in data.columns if c.startswith("fp_")]
if not feature_cols:
    raise ValueError("No fingerprint features (fp_*) found. Check Step 3 output and merging.")

X = data[feature_cols].to_numpy(dtype=np.float32)
y = data["label_active"].to_numpy(dtype=np.int32)

# ### 3) Train Random Forest (fit on all data just to inspect importances)
clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
clf.fit(X, y)

# ### 4) Feature importances and top-20 ranking
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

top_n = 20
top_features = [(feature_cols[i], float(importances[i])) for i in indices[:top_n]]
print("Top 20 features by importance:")
for feat, val in top_features:
    print(f"{feat}: {val:.4f}")

# Save full importance table
feat_import_df = pd.DataFrame({
    "feature": [feature_cols[i] for i in indices],
    "importance": importances[indices].astype(float)
})
feat_import_df.to_csv("step5_feature_importances.csv", index=False)
print("Saved: step5_feature_importances.csv")

# ### 5) Plot cumulative importance
sorted_importances = importances[indices]
cumulative_importance = np.cumsum(sorted_importances)

plt.figure(figsize=(8, 4))
plt.plot(range(1, len(cumulative_importance)+1), cumulative_importance, marker="o")
plt.xlabel("Number of Features")
plt.ylabel("Cumulative Importance")
plt.title("Cumulative Feature Importance (RF - COX2)")
plt.grid(True)
plt.tight_layout()
plt.savefig("step5_cumulative_importance.png", dpi=150)
plt.close()
print("Saved: step5_cumulative_importance.png")

# ### 6) PCA visualization (2D projection)
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(6, 5))
labels = np.unique(y)
for label in labels:
    mask = (y == label)
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1], label=f"Active={int(label)}", alpha=0.7)
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.title("PCA of Fingerprint Features - COX2")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("step5_pca_cox2.png", dpi=150)
plt.close()
print("Saved: step5_pca_cox2.png")

# ### 7) Print PCA explained variance
expl_var = pca.explained_variance_ratio_
print(f"PCA explained variance ratio: PC1={expl_var[0]:.3f}, PC2={expl_var[1]:.3f}")

### 5) Inspect feature importance and a simple PCA view.
### We looked at which fingerprint bits were most important for the COX2 model and did a PCA plot to see how the data looks in two dimensions. 
# The top features have importances between 0.0345 and 0.0121. 
# The PCA shows that the first two components explain about 29% of the variance, so there’s only partial visual separation between active and inactive molecules.


Top 20 features by importance:
fp_530: 0.0345
fp_36: 0.0321
fp_204: 0.0291
fp_599: 0.0239
fp_533: 0.0220
fp_797: 0.0219
fp_627: 0.0201
fp_275: 0.0171
fp_517: 0.0162
fp_846: 0.0154
fp_427: 0.0152
fp_127: 0.0149
fp_544: 0.0138
fp_869: 0.0134
fp_91: 0.0129
fp_1022: 0.0126
fp_299: 0.0124
fp_586: 0.0123
fp_463: 0.0121
fp_865: 0.0121
Saved: step5_feature_importances.csv
Saved: step5_cumulative_importance.png
Saved: step5_pca_cox2.png
PCA explained variance ratio: PC1=0.175, PC2=0.118


In [ ]:
# STEP 6: Proteochemometric (PCM) Model for COX1 and COX2
# Goal: Combine ligand fingerprints + protein amino acid composition features
#       to train a joint model predicting activity across both targets.
# Requirements: pandas, numpy, scikit-learn

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)

# --- 1) Load ligand features (Step 3) ---
lig_path = "step3_ligand_features.csv"
if not os.path.exists(lig_path):
    raise FileNotFoundError(f"Expected ligand file '{lig_path}' not found. Run Step 3 first.")
lig_feats = pd.read_csv(lig_path)  # contains: molecule_chembl_id + fp_* features

# --- 2) Load protein features (Step 2) ---
prot_path = "step2_protein_features.csv"
if not os.path.exists(prot_path):
    raise FileNotFoundError(f"Expected protein file '{prot_path}' not found. Run Step 2 first.")
prot_feats = pd.read_csv(prot_path)  # contains: protein + AA composition features

# --- 3) Load pairs table with pChEMBL values (Step 1) ---
pairs_path = "step1b_pairs_pchembl.csv"
if not os.path.exists(pairs_path):
    raise FileNotFoundError(f"Expected pairs file '{pairs_path}' not found. Run Step 1 first.")
pairs = pd.read_csv(pairs_path)

# --- 4) Define activity labels: Active >= 6.0, else Inactive ---
THRESH = 6.0
pairs["label_active"] = (pairs["pchembl_value"] >= THRESH).astype(int)

# --- 5) Merge ligand + protein features into PCM table ---
df = pairs.merge(lig_feats, on="molecule_chembl_id", how="inner")
df = df.merge(prot_feats, on="protein", how="inner")

# --- 6) Feature matrix and labels ---
feature_cols = [c for c in df.columns if c.startswith("fp_") or len(c) == 1]  # fp_* and AA letters
X = df[feature_cols].to_numpy(dtype=np.float32)
y = df["label_active"].to_numpy(dtype=np.int32)

# Groups: molecule+protein to prevent leakage
groups = df["molecule_chembl_id"].astype(str) + "_" + df["protein"].astype(str)

# --- 7) Group split (80% train / 20% test) ---
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups))

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# --- 8) Train Random Forest ---
clf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# --- 9) Evaluate ---
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()
metrics = {
    "accuracy": round(accuracy_score(y_test, y_pred), 3),
    "precision": round(precision_score(y_test, y_pred), 3),
    "recall": round(recall_score(y_test, y_pred), 3),
    "f1": round(f1_score(y_test, y_pred), 3),
    "roc_auc": round(roc_auc_score(y_test, y_prob), 3) if len(np.unique(y_test)) > 1 else float("nan"),
    "n_train": int(len(y_train)),
    "n_test": int(len(y_test)),
    "confusion_matrix": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
    "threshold_pchembl_active": THRESH
}

print("PCM Model metrics (group split):", metrics)

# --- 10) Save dataset and metrics ---
df_out = df.copy()
df_out["split"] = "train"
df_out.loc[test_idx, "split"] = "test"
df_out.to_csv("step6_pcm_dataset.csv", index=False)
pd.DataFrame([metrics]).to_csv("step6_pcm_metrics.csv", index=False)

print("Saved: step6_pcm_dataset.csv, step6_pcm_metrics.csv")

### is the PCM model that concatenates ligand fingerprints and protein features to predict activity across both targets.
###We trained a proteochemometric (PCM) model combining ligand fingerprints and protein amino acid composition features for both COX1 and COX2. The results show 85% accuracy, high precision (87%), and high recall (90%). 
# This means the model catches most actives while still keeping false positives relatively low. It performs better than the COX2-only model because it uses both ligand and protein information.


PCM Model metrics (group split): {'accuracy': 0.85, 'precision': 0.873, 'recall': 0.899, 'f1': 0.886, 'roc_auc': 0.879, 'n_train': 428, 'n_test': 107, 'confusion_matrix': {'tn': 29, 'fp': 9, 'fn': 7, 'tp': 62}, 'threshold_pchembl_active': 6.0}
Saved: step6_pcm_dataset.csv, step6_pcm_metrics.csv


In [33]:
# STEP 7 — MODEL COMPARISON, SELECTION, AND EXTRA VISUALIZATIONS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve, average_precision_score,
    confusion_matrix, classification_report, brier_score_loss
)
from sklearn.calibration import calibration_curve

# 1) Define models and parameter grids
models_and_params = [
    ("logreg",
     LogisticRegression(max_iter=5000, solver="lbfgs"),
     {"C": [0.01, 0.1, 1, 10]}
    ),
    ("knn",
     Pipeline([("scaler", StandardScaler()), ("clf", KNeighborsClassifier())]),
     {"clf__n_neighbors": [3, 5, 7, 11], "clf__weights": ["uniform", "distance"]}
    ),
    ("rf",
     RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1),
     {"n_estimators": [200, 400], "max_depth": [None, 10, 30], "min_samples_leaf": [1, 3]}
    ),
]

# 2) Grid search with five fold cross validation using ROC AUC
results = []
best_estimators = {}

for name, est, param_grid in models_and_params:
    gs = GridSearchCV(
        estimator=est,
        param_grid=param_grid,
        scoring="roc_auc",
        cv=5,
        n_jobs=-1,
        refit=True,
        verbose=0
    )
    gs.fit(X_train, y_train)
    best_estimators[name] = gs.best_estimator_

    # evaluate on test
    if hasattr(gs.best_estimator_, "predict_proba"):
        y_prob = gs.best_estimator_.predict_proba(X_test)[:, 1]
    else:
        # fallback for models without predict_proba
        y_prob = np.clip(gs.best_estimator_.decision_function(X_test), -20, 20)
        y_prob = (y_prob - y_prob.min()) / (y_prob.max() - y_prob.min() + 1e-12)

    y_pred = (y_prob >= 0.5).astype(int)

    res = {
        "model": name,
        "best_params": str(gs.best_params_),
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_prob),
        "avg_precision": average_precision_score(y_test, y_prob),
    }
    results.append(res)

df_results = pd.DataFrame(results).sort_values("roc_auc", ascending=False)
df_results.to_csv("step7_model_comparison_metrics.csv", index=False)
print(df_results)

# pick the overall best by ROC AUC
best_name = df_results.iloc[0]["model"]
best_model = best_estimators[best_name]
print(f"Best model: {best_name} with params: {best_estimators[best_name]}")

# 3) Bar chart comparing models on ROC AUC and F1
plt.figure()
x = np.arange(len(df_results))
plt.bar(x - 0.2, df_results["roc_auc"].values, width=0.4, label="ROC AUC")
plt.bar(x + 0.2, df_results["f1"].values, width=0.4, label="F1")
plt.xticks(x, df_results["model"].values)
plt.ylabel("Score")
plt.title("Model Comparison")
plt.legend()
plt.tight_layout()
plt.savefig("step7_model_comparison_bar.png", dpi=200)
plt.close()

# 4) Calibration curve and Brier score for best model
if hasattr(best_model, "predict_proba"):
    y_prob_best = best_model.predict_proba(X_test)[:, 1]
else:
    y_dec = np.clip(best_model.decision_function(X_test), -20, 20)
    y_prob_best = (y_dec - y_dec.min()) / (y_dec.max() - y_dec.min() + 1e-12)

frac_pos, mean_pred = calibration_curve(y_test, y_prob_best, n_bins=10, strategy="uniform")
brier = brier_score_loss(y_test, y_prob_best)
print(f"Brier score (lower is better): {brier:.4f}")

plt.figure()
plt.plot([0, 1], [0, 1], linestyle="--")
plt.plot(mean_pred, frac_pos, marker="o")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration Curve")
plt.tight_layout()
plt.savefig("step7_calibration.png", dpi=200)
plt.close()

# 5) Learning curve for best model
train_sizes, train_scores, val_scores = learning_curve(
    best_model, X_train, y_train, cv=5, scoring="roc_auc", n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 6), shuffle=True, random_state=42
)
plt.figure()
plt.plot(train_sizes, train_scores.mean(axis=1), marker="o", label="train")
plt.plot(train_sizes, val_scores.mean(axis=1), marker="o", label="cv")
plt.xlabel("Training examples")
plt.ylabel("ROC AUC")
plt.title("Learning Curve")
plt.legend()
plt.tight_layout()
plt.savefig("step7_learning_curve.png", dpi=200)
plt.close()

# 6) Probability histogram for positive and negative classes
plt.figure()
plt.hist(y_prob_best[y_test==0], bins=20, alpha=0.7, label="COX1")
plt.hist(y_prob_best[y_test==1], bins=20, alpha=0.7, label="COX2")
plt.xlabel("Predicted probability of class 1")
plt.ylabel("Count")
plt.title("Probability Distributions by Class")
plt.legend()
plt.tight_layout()
plt.savefig("step7_probability_hist.png", dpi=200)
plt.close()

# 7) Optional: simple DTI bipartite network from your pairs table
# Uses the pairs DataFrame from step 1 (columns: molecule_chembl_id, protein)
try:
    import networkx as nx
    assert "pairs" in globals(), "pairs DataFrame not found. Load step1_pairs_ic50.csv."
    # If not loaded:
    # pairs = pd.read_csv("step1_pairs_ic50.csv")

    # build graph
    G = nx.Graph()
    drugs = pairs["molecule_chembl_id"].dropna().unique().tolist()
    targets = pairs["protein"].dropna().unique().tolist()  # expects ["COX1","COX2"]

    G.add_nodes_from(drugs, bipartite=0)
    G.add_nodes_from(targets, bipartite=1)

    edges = list(pairs[["molecule_chembl_id", "protein"]].itertuples(index=False, name=None))
    G.add_edges_from(edges)

    deg = dict(G.degree())
    deg_df = pd.DataFrame({"node": list(deg.keys()), "degree": list(deg.values())})
    deg_df.to_csv("step7_network_degrees.csv", index=False)

    # draw a simple layout
    pos = nx.spring_layout(G, seed=42, k=0.3)
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(G, pos, nodelist=targets, node_size=600)
    nx.draw_networkx_nodes(G, pos, nodelist=drugs, node_size=60)
    nx.draw_networkx_edges(G, pos, width=0.5)
    nx.draw_networkx_labels(G, pos, labels={t: t for t in targets}, font_size=10)
    plt.axis("off")
    plt.title("DTI Bipartite Network")
    plt.tight_layout()
    plt.savefig("step7_dti_network.png", dpi=200)
    plt.close()
except Exception as e:
    print("Network figure skipped:", e)


    model                                        best_params  accuracy  \
2      rf  {'max_depth': 30, 'min_samples_leaf': 1, 'n_es...  0.859813   
1     knn  {'clf__n_neighbors': 3, 'clf__weights': 'dista...  0.859813   
0  logreg                                         {'C': 0.1}  0.813084   

   precision    recall        f1   roc_auc  avg_precision  
2   0.875000  0.913043  0.893617  0.878909       0.915117  
1   0.875000  0.913043  0.893617  0.862891       0.885537  
0   0.835616  0.884058  0.859155  0.860984       0.889470  
Best model: rf with params: RandomForestClassifier(max_depth=30, n_estimators=400, n_jobs=-1,
                       random_state=42)
Brier score (lower is better): 0.1254
Network figure skipped: No module named 'networkx'
